## bitMEX trading test

目标：跑通bitMEX交易

策略：专供测试交易系统的随机策略RandomTargetPostionStrategy：每隔30s发出一个随机的SignalEvent。

In [13]:
import logging
import time
import queue

In [ ]:
class NaivePortfolio(object):
    """简单CTA策略的组合管理器
    
    将各个CTA子策略的signal汇总，计算汇总的target_position， (目前是单一identifier)
    当实际仓位与理论仓位不一致时发出OrderEvent，并处理FillEvent。
    """
    
    def __init__(self, event_queue, is_test=True):
        
        self.is_test = True   # 是否测试
        self.logger = logging.getLogger()   # 日志
        
        self.event_queue = event_queue
        
        self.symbol = 'XBTUSD'
        self.lots = 20
        
        # 目标仓位。
        # 当实际仓位与目标仓位不一致时，会触发下单逻辑。
        self.target_position = 0
        
        # 实际仓位。
        # 1. 初始化时会主动查询 self.query_positon()  
        # 2. 收到成交回报时会更新。  -> 如果程序开着时手动下单，那么程序会收到成交回报，把仓位调回到目标仓位。
        self.actual_position = None         
        
    def onSignalEvent(self, signalEvent):
        """
        signalEvent Handler
        
        signalEvent格式:
        {
            identifier: 'Turtle_XBTUSD_1m_9999',
            symbol: 'XBTUSD',
            direction: -1,
        }
        """
        self.logger.info('signalEvent: %s' % signalEvent)
        self._signal_event_handler(signalEvent)   # 处理signal_event
        
    def _signal_event_handler(signalEvent):
        if not signalEvent.get('symbol') == self.symbol:
            self.logger.error('signalEvent.symbol != bitmexTrader.symbol')
            return 
        if signalEvent.get('direction') in (0, -1, 1):
            self.logger.error('invalid signalEvent.direction: %s')
            return
        self.target_position = signalEvent['direction'] * self.lots   # 设定 target_position
        self._trade_to_target()   # 交易
        
    def _trade_to_target(self):
        position_diff = self.target_position - self.actual_position
        if position_diff > 0:
            direction = 'buy'
            limit_price = self.ask1  # 对价下单
        elif position_diff < 0:
            direction = 'sell'
            limit_price = self.bid1   # 对价下单
        else:
            self.logger.info('target==actual, no need to trade')
            return 0
        symbol=self.symbol
        volume=abs(position_diff)
        order_event = orderEvent(symbol, direction, volume, limit_price)   # 构造Order
        
        # TODO: 如果有未执行的委托单怎么办？
        self.event_queue.put(order_event)

重新理清思路：

portfolio 只负责维护 target_position. 发出 target_position_event -> 提示target_position可能有变化。
它的主要功能后续应该是汇总各个子策略的仓位。

executor 负责维护actual_position. 处理发单、撤单、查询持仓。

第一阶段任务，先以最简单的方式，跑通bitMEX实盘交易：
- portofolio线程: 随机生成 target_position 放入队列
- executor线程: 循环从队列中取事件，交易

In [16]:
import threading
import random
import queue


event_q = queue.Queue()  # 事件队列


def generate_random_target_positon(event_queue):
    """随机生成target_position的函数"""
    e = {
        'etype': 'TARGET_POSITION',
        'target_position': random.sample((0, -1, 1), k=1)[0]
    }
    event_queue.put(e)


# 开一个线程，专门生成 target_position event
portfolio_td = threading.Thread(target=generate_random_target_positon, args=(event_q))
portfolio_td.start()

# 另开一个线程，跑executor
bitmex_executor = bitmexExecutor()
executor_td = threading.Thread(target=bitmex_executor.run)
executor_td.start()

# 主程序
while True:
    try:
        event = self.event_queue.get_nowait()
    except queue.Empty:
        time.sleep(0.5)
    else:
        if event.etype == 'TARGET_POSITION':
            bitmex_executor.onTargetPositionEvent(event)   # 交易

In [40]:
class bitmexExecutor(object):
    """bitmex交易执行器"""
    
    def __init__(self):
        print('init bitmexExecutor')
        self.actual_position = self._query_actual_position()   # to-write   
        
    def run(self):
        pass
        
    def onTargetPositionEvent(self):
        pass
    
    def _query_actual_position(self):
        pass    ### 
    
    def _order(self, symbol, side, volume, limit_price=None):
        pass    ### 
        

## Demo of placing an order

**BitMEX 不支持通过 WebSocket 提交或取消委托，这些操作只能通过 HTTP 进行。**

我们的服务器支持保持 HTTP 连接和缓存的 SSL 会话。 如果你保持一个有效连接，你会得到与 websocket 类似的延迟，而无须使用 websocket 进行沟通。

我们保持活动状态的超时时间为90秒。

## bitmexREST

In [4]:
import requests
import datetime
import json
from APIKeyAuthWithExpires import APIKeyAuthWithExpires

class bitmexREST(object):
    """bitmex REST connection"""
    
    def __init__(self, apiKey, apiSecret, is_test=True):
        self.apiKey = apiKey
        self.apiSecret = apiSecret
        self.is_test = is_test
        
        self.base_url = 'https://testnet.bitmex.com/api/v1/' if is_test else ''   # no real trading now
        self.clientOrderID = 0
        
    def _send_http_request(self, verb, path, postdict=None, query=None):
        """send HTTP request"""
        url = self.base_url + path
        auth = APIKeyAuthWithExpires(self.apiKey, self.apiSecret)
        return requests.request(verb, url, json=postdict, params=query, auth=auth)
    
    def place_order(self, symbol, side, qty, limit_price, text=''):
        """place order"""
        
        path = 'order'
        post_dict = {
            'symbol': symbol,
            'side': side,   # 'Buy' or 'Sell'
            'orderQty': qty,
            'price': limit_price,
            'ordType': 'Limit' if limit_price else 'Market',
            'clOrdID': self.clientOrderID,
            'text': self._add_ts(text)
        }
        self.clientOrderID += 1
        return self._send_http_request('POST', path, post_dict)
        
    def cancel_order(self, orderID=None, clOrdID=None, text=''):
        path = 'order'
        post_dict = {
            'orderID': orderID,
            'clOrdID': clOrdID,
            'text': self._add_ts(text)
        }
        return self._send_http_request('DELETE', path, postdict=post_dict)
    
    def cancel_all_orders(self, symbol=None, side=None, text=''):
        path = 'order/all'
        post_dict = {
            'symbol': symbol,
            'filter': json.dumps({'side': side})  # side: 'Buy' or 'Sell'
        }
        return self._send_http_request('DELETE', path, postdict=post_dict)
        
        
    def get_open_orders(self, symbol=None):
        """Get open orders"""
        
        path = 'order'
        query = {
            'symbol': symbol,
            'filter': json.dumps({"ordStatus": "New"})
        }
        return self._send_http_request('GET', path, query=query)
    
    def get_positions(self, symbol=None):
        """Get positions"""
        
        path = 'position'
        query = {}
        if symbol:
            query.update({'filter': json.dumps({'symbol': symbol})})
        return self._send_http_request('GET', path, query=query)
    
    @staticmethod
    def _add_ts(text):
        return '[API][%s] %s' % (str(datetime.datetime.now()), text)

In [11]:
import json


with open('accounts.json') as f:
    acc = json.load(f)

apiKey = acc[0]['apiKey']
apiSecret = acc[0]['apiSecret']

In [2]:
bm = bitmexREST(apiKey, apiSecret)

res = bm.get_positions()
if res.ok:
    print(res.json()[0]['symbol'], res.json()[0]['currentQty'])

res = res = bm.get_open_orders()
if res.ok:
    print(res.json())

res = bm.place_order(symbol='XBTUSD', side='Sell', qty=100, limit_price=None)
if res.ok:
    print(res.json())

res = bm.place_order(symbol='XBTUSD', side='Buy', qty=120, limit_price=6365.0)
if res.ok:
    print(res.json())

## BitMEX WebSocket

copy form https://github.com/BitMEX/api-connectors/ and https://github.com/BitMEX/sample-market-maker

In [1]:
import websocket
import threading
import json
import time
import logging
from APIKeyAuth import generate_nonce, generate_signature


with open('accounts.json') as f:
    acc = json.load(f)

apiKey = acc[0]['apiKey']
apiSecret = acc[0]['apiSecret']
#print(apiKey)
#print(apiSecret)

In [2]:
class bitmexWS(object):
    """bitMEX WebSocket"""
    
    def __init__(self, apiKey=None, apiSecret=None, is_test=True):
        
        self.logger = logging.getLogger()
        self.logger.setLevel(logging.INFO)
        
        self.apiKey = apiKey
        self.apiSecret = apiSecret
        self.shouldAuth = apiKey is not None and apiSecret is not None
        
        self.is_test = is_test
        self.ws_url = 'wss://testnet.bitmex.com/realtime' if is_test else ''
        
        self.ws = None
        self.wst = None
        self.ping_td = None
        self.connected = False
        
    def connect(self):
        self.__connect()
        self.__wait_for_connected()
        self.__start_ping_thread()
        
    def __wait_for_connected(self):
        while not self.connected:
            time.sleep(0.1)
            
    def __start_ping_thread(self):
        self.ping_td = threading.Thread(target=self.__send_ping_forever)
        self.ping_td.start()
            
    def __send_ping_forever(self):
        while self.connected:
            print('>>> seng ping...')
            self.ws.send('ping')
            time.sleep(5)
        
    def exit(self):
        self.connected = False
        self.logger.info('Exiting ...')
        if self.ping_td:
            self.ping_td.join()
            self.logger.info('ping thread end.')
        if self.ws:
            self.ws.close()
        self.logger.info('======================= Exit(intended) =======================')
        
    def __connect(self):
        """connect to websocket in a thread"""
        
        self.ws = websocket.WebSocketApp(self.ws_url,
                                         on_message=self.__on_message,
                                         on_close=self.__on_close,
                                         on_open=self.__on_open,
                                         on_error=self.__on_error,
                                         header=self.__get_auth()
                                        )
        self.wst = threading.Thread(target=lambda: self.ws.run_forever())
        self.wst.start()
        ('ws threading started')
        
    def __get_auth(self):
        """return auth headers"""
        
        if self.shouldAuth is False:
            return []
        
        nonce = generate_nonce()
        return [
            'api-nonce:' + str(nonce),
            'api-signature:' + generate_signature(self.apiSecret, 'GET', '/realtime', nonce, ''),
            'api-key:' + self.apiKey
        ]
    
    def __send_command(self, command, args=None):
        """send a row command"""
        if args is None:
            args = []
        self.ws.send(json.dumps({'op': command, 'args': args}))
        
    def __on_message(self, ws, message):
        """Handler for parsing WS messages"""
        
        print("========================== MESSAGE ==========================")
        print(message)
        
        if message == 'pong':
            return
        
        msg = json.loads(message)
        
        # 1. Welcome info
        if 'info' in msg:
            if msg['info'] == 'Welcome to the BitMEX Realtime API.':
                self.connected = True
                self.logger.info('================ Successful connected to BitMEX WebSocket API ================')
                
        # 2. subscription
        elif 'subscribe' in msg:
            if msg['success']:
                self.logger.info('Subscribe to %s' % msg['subscribe'])
            else:
                self.logger.warn('Subscription not success: %s' % msg)
                
        # 3. table
        elif 'table' in msg:
            self.onData(msg)
        else:
            self.logger.warn('Unclassified msg; %s' % msg)
    
    def onData(self, msg):
        print(msg)   # expected to be overwrite
        
    def __on_error(self, ws, error):
        print('Error: %s' % error)
        
    def __on_close(self, ws):
        print('close')
        
    def __on_open(self, ws):
        print('open')
            
    def subscribe_topic(self, topic):
        # {"op": "subscribe", "args": [<SubscriptionTopic>]}
        self.__send_command('subscribe', [topic])
        

In [3]:
bmws = bitmexWS(apiKey=apiKey, apiSecret=apiSecret)
bmws.connect()

bmws.subscribe_topic('order')   # manual send orders and make fill events on website
bmws.subscribe_topic('instrument:XBTUSD')

open
========================== MESSAGE ==========================
{"info":"Welcome to the BitMEX Realtime API.","version":"2018-09-07T19:34:23.000Z","timestamp":"2018-09-11T09:01:14.091Z","docs":"https://testnet.bitmex.com/app/wsAPI","limit":{"remaining":39}}
>>> seng ping...
========================== MESSAGE ==========================
pong
========================== MESSAGE ==========================
{"table":"order","action":"partial","keys":["orderID"],"types":{"orderID":"guid","clOrdID":"symbol","clOrdLinkID":"symbol","account":"long","symbol":"symbol","side":"symbol","simpleOrderQty":"float","orderQty":"long","price":"float","displayQty":"long","stopPx":"float","pegOffsetValue":"float","pegPriceType":"symbol","currency":"symbol","settlCurrency":"symbol","ordType":"symbol","timeInForce":"symbol","execInst":"symbol","contingencyType":"symbol","exDestination":"symbol","ordStatus":"symbol","triggered":"symbol","workingIndicator":"boolean","ordRejReason":"symbol","simpleLeavesQty":"f

In [4]:
bmws.exit()

========================== MESSAGE ==========================
{"table":"instrument","action":"update","data":[{"symbol":"XBTUSD","impactMidPrice":6329,"impactAskPrice":6335.1283,"timestamp":"2018-09-11T09:01:19.100Z"}]}
{'table': 'instrument', 'data': [{'impactMidPrice': 6329, 'symbol': 'XBTUSD', 'impactAskPrice': 6335.1283, 'timestamp': '2018-09-11T09:01:19.100Z'}], 'action': 'update'}
close


## Notes

订阅order主题，接收到的table